In [39]:
# import packages
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [18]:
erawijantari_x_path = 'data/salma/erawijantari_x.csv'
erawijantari_y_path = 'data/salma/erawijantari_y.csv'

In [19]:
erawijantari_x = pd.read_csv(erawijantari_x_path,)
erawijantari_y = pd.read_csv(erawijantari_y_path,)

In [14]:
erawijantari_x.columns

Index(['d__Bacteria;p__Firmicutes_A;c__Clostridia;o__Oscillospirales;f__Oscillospiraceae;g__F23-B02;s__F23-B02 sp900556535',
       'd__Bacteria;p__Firmicutes_A;c__Clostridia;o__Oscillospirales;f__Oscillospiraceae;g__CAG-103;s__CAG-103 sp018265835',
       'd__Bacteria;p__Firmicutes_A;c__Clostridia;o__Oscillospirales;f__Oscillospiraceae;g__ER4;s__ER4 sp900556145',
       'd__Bacteria;p__Firmicutes_A;c__Clostridia;o__Oscillospirales;f__Oscillospiraceae;g__F23-B02;s__F23-B02 sp900772725',
       'd__Bacteria;p__Firmicutes_A;c__Clostridia;o__Oscillospirales;f__Oscillospiraceae;g__NSJ-62;s__NSJ-62 sp018919205',
       'd__Bacteria;p__Firmicutes_A;c__Clostridia;o__Oscillospirales;f__Oscillospiraceae;g__ER4;s__ER4 sp017527825',
       'd__Bacteria;p__Firmicutes_A;c__Clostridia;o__Oscillospirales;f__Oscillospiraceae;g__CAG-103;s__CAG-103 sp902760845',
       'd__Bacteria;p__Firmicutes_A;c__Clostridia;o__Oscillospirales;f__Ruminococcaceae;g__Harryflintia;s__Harryflintia acetispora',
       'd_

In [15]:
erawijantari_x.shape

(96, 40)

In [20]:
erawijantari_y.columns

Index(['Study.Group'], dtype='object')

In [21]:
erawijantari_y.shape

(96, 1)

In [24]:
erawijantari_y


,Study.Group
0,Healthy
1,Healthy
2,Healthy
3,Healthy
4,Healthy
...,...
91,Gastrectomy
92,Healthy
93,Healthy
94,Gastrectomy


In [25]:
# Assuming erawijantari_x and erawijantari_y are pandas DataFrame
X = erawijantari_x.values
y = erawijantari_y['Study.Group'].values

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Convert to tensors
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y_encoded, dtype=torch.long)

# Split data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.2, random_state=42)

# Create TensorDatasets
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [31]:
import torch.nn as nn

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.layer1 = nn.Linear(40, 64)
        # self.layer2 = nn.Linear(128, 64)
        self.output_layer = nn.Linear(64, 2)
    
    def forward(self, x):
        x = torch.relu(self.layer1(x))
        # x = torch.relu(self.layer2(x))
        x = self.output_layer(x)
        return x

model = NeuralNetwork()


In [32]:
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [33]:
epochs = 100

for epoch in range(epochs):
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()
    
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

# Optionally, after each epoch, evaluate your model on the test data


Epoch 1, Loss: 281.31744384765625
Epoch 2, Loss: 176.9986114501953
Epoch 3, Loss: 167.2524871826172
Epoch 4, Loss: 11.832443237304688
Epoch 5, Loss: 90.53738403320312
Epoch 6, Loss: 19.380109786987305
Epoch 7, Loss: 52.23789596557617
Epoch 8, Loss: 44.07072830200195
Epoch 9, Loss: 148.75857543945312
Epoch 10, Loss: 1.5894556781859137e-07
Epoch 11, Loss: 26.7750301361084
Epoch 12, Loss: 1.0794806480407715
Epoch 13, Loss: 69.40055084228516
Epoch 14, Loss: 2.8588428497314453
Epoch 15, Loss: 18.98358726501465
Epoch 16, Loss: 11.1744384765625
Epoch 17, Loss: 44.356136322021484
Epoch 18, Loss: 1.4741348028182983
Epoch 19, Loss: 5.401528835296631
Epoch 20, Loss: 21.573776245117188
Epoch 21, Loss: 0.0
Epoch 22, Loss: 0.28603458404541016
Epoch 23, Loss: 18.76792335510254
Epoch 24, Loss: 0.49804040789604187
Epoch 25, Loss: 15.418559074401855
Epoch 26, Loss: 0.004103260580450296
Epoch 27, Loss: 11.00326919555664
Epoch 28, Loss: 0.07292751222848892
Epoch 29, Loss: 0.02639959193766117
Epoch 30, Los

In [34]:
def test_model(model, test_loader):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0

    with torch.no_grad():  # During testing, gradient computation is unnecessary
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Accuracy of the model on the test set: {accuracy:.2f}%')


In [30]:
# Training loop as defined earlier
epochs = 100
for epoch in range(epochs):
    model.train()  # Make sure the model is in training mode
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()
    
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

# Testing loop
test_model(model, test_loader)


Epoch 1, Loss: 0.005405260715633631
Epoch 2, Loss: 8.222163887694478e-05
Epoch 3, Loss: 1.5993764463928528e-06
Epoch 4, Loss: 3.973642037635727e-08
Epoch 5, Loss: 0.0001193355637951754
Epoch 6, Loss: 0.0002158268471248448
Epoch 7, Loss: 0.00011978172551607713
Epoch 8, Loss: 0.00015553459525108337
Epoch 9, Loss: 0.00010642511915648356
Epoch 10, Loss: 1.4801688621446374e-06
Epoch 11, Loss: 0.0006191019783727825
Epoch 12, Loss: 1.336030982201919e-05
Epoch 13, Loss: 4.4860313209937885e-05
Epoch 14, Loss: 1.7670918168732896e-05
Epoch 15, Loss: 0.0007006505038589239
Epoch 16, Loss: 1.4056113286642358e-05
Epoch 17, Loss: 2.524867704778444e-05
Epoch 18, Loss: 0.001168683054856956
Epoch 19, Loss: 0.00015856580284889787
Epoch 20, Loss: 9.983417839976028e-06
Epoch 21, Loss: 6.953872144777051e-08
Epoch 22, Loss: 1.797894037736114e-05
Epoch 23, Loss: 0.0006497439462691545
Epoch 24, Loss: 0.00011592932423809543
Epoch 25, Loss: 0.002313216682523489
Epoch 26, Loss: 0.00015643447113689035
Epoch 27, Los

In [37]:
# Assuming erawijantari_x and erawijantari_y are loaded as described previously

# Preprocessing
# Convert categorical output to numeric
encoder = LabelEncoder()
y = encoder.fit_transform(erawijantari_y['Study.Group'])

# Scale the input features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(erawijantari_x)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

# Convert arrays to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
y_test = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

# DataLoader
train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=16, shuffle=True)

# Model Design
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.layer1 = nn.Linear(X_train.shape[1], 128)
        self.layer2 = nn.Linear(128, 64)
        self.layer3 = nn.Linear(64, 32)
        self.layer4 = nn.Linear(32, 1)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        x = self.relu(self.layer1(x))
        x = self.dropout(x)
        x = self.relu(self.layer2(x))
        x = self.dropout(x)
        x = self.relu(self.layer3(x))
        x = self.dropout(x)
        x = self.sigmoid(self.layer4(x))
        return x

# Create the model
model = NeuralNetwork()

# Loss and Optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training Loop
epochs = 50
for epoch in range(epochs):
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}')

# Evaluation
with torch.no_grad():
    y_pred = model(X_test)
    y_pred = y_pred.round()
    accuracy = (y_pred == y_test).float().mean()
    print(f'Test Accuracy: {accuracy:.4f}')


Epoch 1/50, Loss: 0.6231
Epoch 2/50, Loss: 0.6598
Epoch 3/50, Loss: 0.6264
Epoch 4/50, Loss: 0.6233
Epoch 5/50, Loss: 0.6011
Epoch 6/50, Loss: 0.5481
Epoch 7/50, Loss: 0.6931
Epoch 8/50, Loss: 0.4518
Epoch 9/50, Loss: 0.6117
Epoch 10/50, Loss: 0.6006
Epoch 11/50, Loss: 0.2609
Epoch 12/50, Loss: 0.2892
Epoch 13/50, Loss: 0.1310
Epoch 14/50, Loss: 0.3257
Epoch 15/50, Loss: 0.3517
Epoch 16/50, Loss: 0.2997
Epoch 17/50, Loss: 0.0098
Epoch 18/50, Loss: 0.1336
Epoch 19/50, Loss: 0.2205
Epoch 20/50, Loss: 0.0935
Epoch 21/50, Loss: 0.1963
Epoch 22/50, Loss: 0.1043
Epoch 23/50, Loss: 0.0640
Epoch 24/50, Loss: 0.1425
Epoch 25/50, Loss: 0.0194
Epoch 26/50, Loss: 0.0202
Epoch 27/50, Loss: 0.1559
Epoch 28/50, Loss: 0.0994
Epoch 29/50, Loss: 0.0045
Epoch 30/50, Loss: 0.0377
Epoch 31/50, Loss: 0.0019
Epoch 32/50, Loss: 0.0040
Epoch 33/50, Loss: 0.0367
Epoch 34/50, Loss: 0.0611
Epoch 35/50, Loss: 0.0001
Epoch 36/50, Loss: 0.0450
Epoch 37/50, Loss: 0.0621
Epoch 38/50, Loss: 0.0010
Epoch 39/50, Loss: 0.

In [42]:
kim_adenomas_X_path = 'data/mohammad/kim_adenomas_X.csv'
kim_adenomas_y_path = 'data/mohammad/kim_adenomas_y.csv'
kim_adenomas_X = pd.read_csv(kim_adenomas_X_path,)
kim_adenomas_y = pd.read_csv(kim_adenomas_y_path,)

In [43]:
# Assuming erawijantari_x and erawijantari_y are loaded as described previously

# Preprocessing
# Convert categorical output to numeric
encoder = LabelEncoder()
y = encoder.fit_transform(kim_adenomas_y['Study.Group'])

# Scale the input features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(kim_adenomas_X)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

# Convert arrays to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
y_test = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

# DataLoader
train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=16, shuffle=True)

# Model Design
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.layer1 = nn.Linear(X_train.shape[1], 128)
        self.layer2 = nn.Linear(128, 64)
        self.layer3 = nn.Linear(64, 32)
        self.layer4 = nn.Linear(32, 1)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        x = self.relu(self.layer1(x))
        x = self.dropout(x)
        x = self.relu(self.layer2(x))
        x = self.dropout(x)
        x = self.relu(self.layer3(x))
        x = self.dropout(x)
        x = self.sigmoid(self.layer4(x))
        return x

# Create the model
model = NeuralNetwork()

# Loss and Optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training Loop
epochs = 50
for epoch in range(epochs):
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}')

# Evaluation
with torch.no_grad():
    y_pred = model(X_test)
    y_pred = y_pred.round()
    accuracy = (y_pred == y_test).float().mean()
    print(f'Test Accuracy: {accuracy:.4f}')


Epoch 1/50, Loss: 0.7061
Epoch 2/50, Loss: 0.6890
Epoch 3/50, Loss: 0.6821
Epoch 4/50, Loss: 0.6846
Epoch 5/50, Loss: 0.6452
Epoch 6/50, Loss: 0.7930
Epoch 7/50, Loss: 0.6944
Epoch 8/50, Loss: 0.6287
Epoch 9/50, Loss: 0.6429
Epoch 10/50, Loss: 0.5631
Epoch 11/50, Loss: 0.5885
Epoch 12/50, Loss: 0.6495
Epoch 13/50, Loss: 0.6983
Epoch 14/50, Loss: 0.5042
Epoch 15/50, Loss: 0.5725
Epoch 16/50, Loss: 0.3359
Epoch 17/50, Loss: 0.5526
Epoch 18/50, Loss: 0.6517
Epoch 19/50, Loss: 0.4777
Epoch 20/50, Loss: 0.5509
Epoch 21/50, Loss: 0.3756
Epoch 22/50, Loss: 0.4308
Epoch 23/50, Loss: 0.4109
Epoch 24/50, Loss: 0.4719
Epoch 25/50, Loss: 0.3515
Epoch 26/50, Loss: 0.4473
Epoch 27/50, Loss: 0.6464
Epoch 28/50, Loss: 0.5940
Epoch 29/50, Loss: 0.3717
Epoch 30/50, Loss: 0.6188
Epoch 31/50, Loss: 0.4659
Epoch 32/50, Loss: 0.8280
Epoch 33/50, Loss: 0.5914
Epoch 34/50, Loss: 0.3825
Epoch 35/50, Loss: 0.9047
Epoch 36/50, Loss: 0.2476
Epoch 37/50, Loss: 0.4669
Epoch 38/50, Loss: 0.3720
Epoch 39/50, Loss: 0.